In [30]:
# Importing necessary libraries
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
import pandas as pd
import math
from numpy import matrix

In [31]:
# Read in the playlists dataset
data = pd.read_csv('datasets/data.csv')
data

,Unnamed: 0,name,artists,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,nlp_lyrics,nlp_annotations,valence+nlp,mood
0,0,Honesty,['Pink Sweat$'],0.9190,0.616,191000,0.204,0.000000,6,0.1060,-11.421,0,0.0331,79.826,4,0.419,0.9490,0.9490,0.4380,Sad
1,1,Get You (feat. Kali Uchis),"['Daniel Caesar', 'Kali Uchis']",0.4260,0.659,277297,0.295,0.000069,11,0.0795,-8.528,0,0.0322,74.037,4,0.360,0.6124,0.6124,0.3722,Sad
2,2,Easy,['Mac Ayres'],0.3730,0.722,314358,0.282,0.005570,5,0.1470,-7.484,1,0.1060,91.367,4,0.217,0.9933,0.9933,0.2369,Sad
3,3,Feel No Ways,['Drake'],0.0312,0.582,240653,0.677,0.000007,7,0.1220,-7.081,1,0.1720,193.818,4,0.205,0.9310,0.9310,0.2236,Sad
4,4,Want U Around (feat. Ruel),"['Omar Apollo', 'Ruel']",0.3000,0.571,247720,0.443,0.000007,0,0.1010,-6.760,1,0.0451,143.923,3,0.621,0.9123,0.9123,0.6392,Sad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3950,95,Back for Good - Radio Mix,['Take That'],0.3100,0.601,242267,0.504,0.000000,5,0.1370,-9.761,1,0.0284,78.986,4,0.564,0.9853,0.9853,0.5837,Aroused
3951,96,How Do I Live,['LeAnn Rimes'],0.1280,0.577,266973,0.462,0.000000,2,0.0822,-7.989,1,0.0279,128.303,4,0.258,0.4203,0.4203,0.2664,Aroused
3952,97,Drops of Jupiter (Tell Me),['Train'],0.1530,0.481,259933,0.638,0.000000,0,0.1540,-5.862,1,0.0276,79.064,4,0.497,0.9870,0.9870,0.5167,Aroused
3953,98,You're Still The One,['Shania Twain'],0.4760,0.556,212560,0.513,0.000000,3,0.2370,-7.201,1,0.0290,133.803,4,0.665,0.9953,0.9953,0.6849,Aroused


In [32]:
# Pre-process the data

# Remove the index column from csv file
data = data.loc[:, ~data.columns.str.contains('^Unnamed')] 

# Drop unnecessary columns
df = data.drop(columns=['duration_ms', 'nlp_lyrics', 'nlp_annotations'])

# Randomise rows
df = df.sample(frac=1).reset_index(drop=True)

df

,name,artists,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,valence+nlp,mood
0,Still,['Sonic Being'],0.937000,0.0607,0.00396,0.71600,6,0.1090,-30.216,0,0.0490,69.776,4,0.07020,0.05020,Sleepy
1,Bolero Chiquito,['Eu Nice'],0.991000,0.4010,0.08140,0.95000,1,0.1040,-25.158,1,0.0322,78.553,4,0.17500,0.17500,Calm
2,Mind Of Lust,['Sandomina'],0.925000,0.3110,0.15300,0.93300,8,0.0997,-23.937,1,0.0309,87.040,4,0.03390,0.01390,Calm
3,20 Joints,['Berner'],0.251000,0.8360,0.73100,0.00000,6,0.0679,-7.396,0,0.1460,100.861,4,0.76600,0.76120,Angry
4,Hyperthermic / Accelerate,['Lamb of God'],0.000012,0.2670,0.99600,0.38600,2,0.0503,-3.385,1,0.0819,104.916,3,0.06480,0.04490,Angry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3950,Remedies,"['Mollie Collins', 'Ruth Royall']",0.017200,0.4770,0.58100,0.00000,8,0.1090,-8.765,0,0.0569,85.375,4,0.04730,0.06120,Energised
3951,Thunder & Pouring Rain,['ABC Tonnerre'],0.000068,0.2070,0.73000,0.98700,4,0.3790,-17.295,0,0.0617,96.966,4,0.00282,0.00282,Sleepy
3952,Framandi,['Syfjaður'],0.939000,0.1040,0.05780,0.92600,7,0.1020,-26.086,1,0.0417,68.408,4,0.05140,0.05710,Sleepy
3953,Swervin (feat. 6ix9ine),"['A Boogie Wit da Hoodie', '6ix9ine']",0.015300,0.5810,0.66200,0.00000,9,0.1110,-5.239,1,0.3030,93.023,4,0.43400,0.43760,Energised


In [33]:
X = df.iloc[:, 2:15] # All rows, features only, no labels
y = df.iloc[:, 15] # All rows, label only, no features

In [34]:
# create loocv procedure
cv = LeaveOneOut()
# enumerate splits
y_true, y_pred = list(), list()

In [35]:
from sklearn.svm import SVC


In [56]:
for train_ix, test_ix in cv.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix, :], X.iloc[test_ix, :]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    # fit model
    model = SVC(kernel = 'linear', C = 1)
    model.fit(X_train, y_train)
    # evaluate model
    yhat = model.predict(X_test)
    print(y_test)
    print(yhat)
    # store
    y_true.append(y_test.iloc[0])
    y_pred.append(yhat[0])

0    Sleepy
Name: mood, dtype: object
['Sleepy']
1    Calm
Name: mood, dtype: object
['Calm']
2    Calm
Name: mood, dtype: object
['Calm']
3    Angry
Name: mood, dtype: object
['Happy']
4    Angry
Name: mood, dtype: object
['Angry']
5    Energised
Name: mood, dtype: object
['Angry']
6    Happy
Name: mood, dtype: object
['Energised']
7    Happy
Name: mood, dtype: object
['Happy']
8    Chill
Name: mood, dtype: object
['Sad']
9    Sad
Name: mood, dtype: object
['Angry']
10    Sleepy
Name: mood, dtype: object
['Sleepy']
11    Happy
Name: mood, dtype: object
['Happy']
12    Happy
Name: mood, dtype: object
['Energised']
13    Energised
Name: mood, dtype: object
['Energised']
14    Calm
Name: mood, dtype: object
['Calm']
15    Happy
Name: mood, dtype: object
['Energised']
16    Sad
Name: mood, dtype: object
['Chill']
17    Sleepy
Name: mood, dtype: object
['Sleepy']
18    Angry
Name: mood, dtype: object
['Energised']
19    Aroused
Name: mood, dtype: object
['Energised']
20    Calm
Name: mood,

In [64]:
def mood(mood):
        
    if mood == 'Happy':
        mood = 0
    elif mood == 'Sad':
        mood = 1
    elif mood == 'Calm':
        mood = 2
    elif mood == 'Sleepy':
        mood = 3
    elif mood == 'Energised':
        mood = 4
    elif mood == 'Aroused':
        mood = 5
    elif mood == 'Angry':
        mood = 6
    elif mood == 'Chill':
        mood = 7
    
    return mood

In [67]:
def similarity(predictions, y_test):
    similarity = [
    [ 1,  0, .5, .3, .5, .7,  0, .6],
    [ 0,  1, .5, .4, .3, .5, .7, .5],
    [.5, .5,  1, .8,  0, .5,  0, .9],
    [.3, .4, .8,  1,  0, .2,  0, .5],
    [.5, .3,  0,  0,  1, .3, .8,  0],
    [.7, .5, .5, .2, .3,  1, .2, .6],
    [ 0, .7,  0,  0, .8, .2,  1,  0],
    [.6, .5, .9, .5,  0, .6,  0,  1]
    ]
    sum_similarity = 0

    for row in range(len(predictions)):
        model_mood = mood(predictions[row])
        y_mood = mood(y_test[row])
        sum_similarity += similarity[model_mood][y_mood]
    
    accuracy = sum_similarity/len(predictions)
    return accuracy



In [68]:
c_accuracy = similarity(y_pred, y_true)
c_accuracy

0.8169906589245146